In [14]:
import os
os.environ['HF_HOME'] = '/workspace/cache/huggingface/'
os.environ['HF_HOME'] = '/workspace/cache/huggingface/'
os.chdir('/workspace/FutureGPT2/src/')

import numpy as np
from torch import optim, nn, Tensor
from torch.nn import functional as F
import torch
import wandb
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
import transformers
from inspect import signature, _ParameterKind
import copy
import gc
import datasets
from torch.utils.data import DataLoader
from datasets import load_dataset
from matplotlib import pyplot as plt
from itertools import islice
import random
from models.gpt_model import *
from data.mmap_dataset import MMapIndexedDataset

import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import WandbLogger

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
if torch.cuda.get_device_capability()[0] >= 8:
    torch.set_float32_matmul_precision('high')

In [3]:
wandb.login(key='os.environ[WANDB_API_KEY]', relogin=True)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/wwu/.netrc


True

In [4]:
# https://github.com/EleutherAI/pythia
LR_DICT = {
    '14m':  1.0e-3,
    '70m':  1.0e-3,
    '160m': 6.0e-4,
    '410m': 3.0e-4,
    '1b':   3.0e-4,
    '1.4b': 2.0e-4,
    '2.8b': 1.6e-4,
    '6.9b': 1.2e-4,
    '12b':  1.2e-4,
}

model_size = '70m'
params = {
    'model_name': f'EleutherAI/pythia-{model_size}-deduped',
    'lr': LR_DICT[model_size]
}

In [19]:
PILE_PATH = '/workspace/corpus/pythia_pile_idxmaps/pile_0.87_deduped_text_document.bin'
train = MMapIndexedDataset(PILE_PATH, skip_warmup=True)

    reading sizes...
    reading pointers...
    reading document index...
    creating numpy buffer of mmap...
    creating memory view of numpy buffer...


In [34]:
''.join(Token[i] for i in train[10]).replace('Ġ', ' ')

"Playing back a meeting recordingĊĊâĢ¦Let me show you how to locate and play back a meeting that you have recorded.âĢ¦First, let's understand how WebEx Meetings store and prepare your meeting recordings.âĢ¦The meetings are recorded on the WebEx server.âĢ¦WebEx will post the recording to theirâĢ¦server within 24 hours of the meeting completion.âĢ¦When your recording is ready, you'll receive an update onâĢ¦your dashboard homepage with the playback link and the recording information.âĢ¦Let me show you how that looks.âĢ¦When you get this notification, you can click the link that says Play Recording.âĢ¦And WebEx will play back the video for you with the WebEx network recording player.âĢ¦ĊĊTo locate your meeting recording manually, ifâĢ¦you miss the notification, the easiest thingâĢ¦to do is look at the meetings space for the meeting that you recorded.âĢ¦First, find the meeting in your meetings list by clicking the Meetings tab.âĢ¦Click the Recent tab.âĢ¦You'll note, in the list, whether it'

In [5]:
# 5M examples sampled from the-pile. truncated to len 64
train = load_dataset(
    'EleutherAI/pile-deduped-pythia-random-sampled', 
    split='train'
)
train = train.rename_column('Tokens', 'input_ids')
train = train.remove_columns([c for c in train.column_names if c != 'input_ids'])
train = train.cast_column('input_ids', datasets.Sequence(datasets.Value('int64')))
train = train.with_format('torch')
train_loader = DataLoader(train, batch_size=512, num_workers=96)

In [6]:
NAME = '_'.join(
    [f'PYTHIA-VANILLA'] + 
    [f'{k}-{v}' for k, v in {**params}.items()]
).replace('EleutherAI/', '')
PROJ = 'LAISR_FUTURE_PYTHIA'
wandb_logger = WandbLogger(
    name=NAME,
    project=PROJ,
    log_model=False,   # Only save checkpoints locally
)

In [7]:
lr_monitor = LearningRateMonitor()
checkpoint_callback = ModelCheckpoint(
    dirpath="/workspace/checkpoints",
    filename=NAME + "_{global_step}_{val_loss:.2f}",
    every_n_epochs=1,
    save_top_k=1,
    monitor='val_loss',
    mode='min',
)
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    divergence_threshold=15,
    min_delta=0.00,
    patience=10,
    verbose=False,
    mode='min',
)
trainer = L.Trainer(
    fast_dev_run=False,
    logger=wandb_logger,
    #val_check_interval=.1,
    callbacks=[checkpoint_callback, early_stop_callback, lr_monitor],
    max_epochs=1,
    enable_progress_bar=True,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [8]:
model = LitGPTModel(**params)
wandb_logger.watch(model.model, log='all', log_graph=False)

wandb: Currently logged in as: wilswu. Use `wandb login --relogin` to force relogin


In [9]:
trainer.fit(
    model=model,
    train_dataloaders=train_loader,
)

/home/wwu/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/home/wwu/.local/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:639: Checkpoint directory /workspace/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name  | Type               | Params
---------------------------------------------
0 | model | GPTNeoXForCausalLM | 70.4 M
---------------------------------------------
70.4 M    Trainable params
0         Non-trainable params
70.4 M    Total params
281.706   Total estimated model params size (MB)


NUM TRAINING STEPS 9766


Training: |          | 0/? [00:00<?, ?it/s]

/home/wwu/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
Process Process-35:
Process Process-49:
Process Process-41:
Process Process-43:
Process Process-40:
Process Process-44:
Process Process-38:
Process Process-42:
Process Process-48:
Process Process-37:
Process Process-46:
Process Process-36:
Process Process-34:
Process Process-45:
Process Process-47:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 317, in _bootstrap
    util._exit_function()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 317, in _bootstrap
    util._exit_function()
Traceback (most recent call